In [2]:
import mlflow
import torch 
import torch.nn as nn 
import numpy as np 
import datetime
import ast
import pickle

In [3]:
from ExchnageEnv import MarketEnvironment

In [4]:
''' 
sharpe ratio measures the excess return of the portfolio over the 
volatility of it -> risk adjusted performance
'''

def sharp_ratio_(rewards, tran_costs):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

In [5]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f"torch is running on : {device}")

torch is running on : cuda


In [6]:
mlflow_tracking_uri = 'file:/home/naradaw/dev/Charles_Schwab/code/RnD/v3/mlflow_experiments'
mlflow.set_tracking_uri(mlflow_tracking_uri)

experiment_name = "/portfolio-contructor-v3"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///home/naradaw/dev/Charles_Schwab/code/RnD/v3/mlflow_experiments/168050635922118841', creation_time=1730954894954, experiment_id='168050635922118841', last_update_time=1730954894954, lifecycle_stage='active', name='/portfolio-contructor-v3', tags={}>

In [7]:
data_path = "/home/naradaw/dev/Charles_Schwab/data/w_features/v1/2024_10_31/dataset_sqs_2024_10_31_11_18.pkl"

In [8]:
with open(data_path, 'rb') as f:
    data_dict = pickle.load(f)

In [9]:
mlflow_tracking_uri = '/home/naradaw/dev/Charles_Schwab/code/RnD/v3/mlflow_experiments'
mlflow_registry_uri = '/home/naradaw/dev/Charles_Schwab/code/RnD/v3/mlflow_model_registry'

In [10]:
mlflow_client = mlflow.client.MlflowClient(
    tracking_uri=mlflow_tracking_uri,
    registry_uri= mlflow_registry_uri
)

In [11]:
run_id = '17ce97946d7e47c1996f5e06b7853cf0'

In [12]:
run = mlflow_client.get_run(
    run_id=run_id
)

In [13]:
run_params = run.data.params
run_params

{'feature_set': "['close', 'volume_mfi', 'volume_fi', 'volume_nvi', 'trend_visual_ichimoku_a', 'trend_aroon_up', 'volatility_atr', 'trend_psar_down_indicator', 'volatility_bbl', 'volatility_dcl', 'trend_vortex_ind_neg', 'trend_cci', 'volume_sma_em', 'volatility_kch', 'volatility_kcl', 'trend_aroon_ind', 'volume_cmf', 'volatility_kcp', 'trend_ichimoku_base', 'momentum_stoch_rsi_d', 'momentum_ppo_signal', 'trend_ema_slow', 'volatility_bbhi', 'momentum_wr', 'trend_psar_up', 'volume_obv', 'others_dr', 'momentum_stoch', 'momentum_ppo', 'volatility_bbh', 'volatility_kcw', 'trend_adx_neg', 'volatility_dcm', 'trend_psar_up_indicator', 'momentum_pvo_signal', 'momentum_ppo_hist', 'trend_kst_diff', 'volume_vwap', 'volatility_bbm', 'momentum_uo', 'volatility_dcp', 'momentum_pvo_hist', 'trend_vortex_ind_diff', 'others_dlr', 'momentum_stoch_rsi', 'trend_macd_signal', 'trend_macd', 'momentum_kama', 'trend_dpo', 'others_cr', 'trend_visual_ichimoku_b', 'trend_kst', 'trend_adx', 'momentum_stoch_signal',

In [14]:
market_env = MarketEnvironment(
        device = device,
        data_path = data_path,
        holding_period = int(run_params['holding_period']),
        episode_duration = int(run_params['episode_duration']),
        train_test_split = float(run_params['train_test_split']),
        symbol_universe = ast.literal_eval(run_params['symbol_universe']),
        feature_set = ast.literal_eval(run_params['feature_set'])
        )

In [15]:
logged_model = 'runs:/17ce97946d7e47c1996f5e06b7853cf0/portfolio_constructor_2024_11_11_09_52'
# Load model as a PyFuncModel.
loaded_model = mlflow.pytorch.load_model(logged_model)

In [16]:
is_end = False
returns_ = []
symbol_universe_returns = []
tran_costs = []

market_env.reset(mode = "test")
state = market_env.get_state()



In [17]:
market_env.split_, market_env.t_, market_env.end_t_

(933, 933, 1173)

In [18]:
symbol_universe = ast.literal_eval(run_params['symbol_universe'])

In [19]:
symbol_universe_allocations = torch.Tensor(np.repeat(1/len(symbol_universe), len(symbol_universe))).to(device)
symbol_universe_allocations


tensor([0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500,
        0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500,
        0.0500, 0.0500], device='cuda:0')

In [20]:
while not is_end:
    _, allocations = loaded_model(state)

    symbol_universe_return_= market_env.get_baseline_return(symbol_universe_allocations)
    state, return_, is_end, tran_cost = market_env.step(allocations)
    
    symbol_universe_returns.append(symbol_universe_return_)
    returns_.append(return_)
    tran_costs.append(tran_cost)

sharp_ratio = sharp_ratio_(returns_, tran_costs)
sharp_ratio

tensor(0.8776, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

In [21]:
uni_sharp_ratio = sharp_ratio_(symbol_universe_returns, tran_costs)
uni_sharp_ratio

tensor(0.8643, device='cuda:0', dtype=torch.float64)

In [23]:
returns_

[tensor(518.7401, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(375.3623, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(189.0472, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(627.5846, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(594.6323, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(39.2412, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(30.9920, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(-416.4769, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(-250.9532, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(831.2970, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(469.2236, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(930.9438, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(459.0675, device='cu

In [22]:
symbol_universe_returns

[tensor(337.5375, device='cuda:0', dtype=torch.float64),
 tensor(571.0625, device='cuda:0', dtype=torch.float64),
 tensor(8.4322, device='cuda:0', dtype=torch.float64),
 tensor(415.6478, device='cuda:0', dtype=torch.float64),
 tensor(533.1890, device='cuda:0', dtype=torch.float64),
 tensor(52.1050, device='cuda:0', dtype=torch.float64),
 tensor(-1.9826, device='cuda:0', dtype=torch.float64),
 tensor(-330.1436, device='cuda:0', dtype=torch.float64),
 tensor(51.0758, device='cuda:0', dtype=torch.float64),
 tensor(313.4208, device='cuda:0', dtype=torch.float64),
 tensor(357.6756, device='cuda:0', dtype=torch.float64),
 tensor(589.2537, device='cuda:0', dtype=torch.float64),
 tensor(76.3052, device='cuda:0', dtype=torch.float64)]